In [57]:
from bs4 import BeautifulSoup
from os import listdir
import regex as re
import tqdm
import pickle

In [55]:
path = "data/nos/feeds.nos.nl/nosnieuwsalgemeen"

In [ ]:
def process_page(file):
    page_dict = {}

    with open(file, "r", encoding = "utf8") as page: 
        texts = []
        tok_count = 0
        soup = BeautifulSoup(page, "lxml")
        content = soup.find_all("description")
        for piece in content[1:]:
            text = piece.get_text()
            text = re.sub(">|}|]", "", text)
            if text != "":
                texts.append(text)
                tok_count += len(text.split(" "))
        
        page_dict["tokens"] = tok_count
        page_dict["docs"] = len(texts)
        page_dict["texts"] = texts
    
    return page_dict if page_dict["texts"] != [] else None

def combine(dict_a, dict_b):
    dict_a["tokens"] += dict_b["tokens"]
    dict_a["docs"] += dict_b["docs"]
    dict_a["texts"] += dict_b["texts"]
    return dict_a

def process_dir(path):
    files = [path + "/" +file for file in listdir(path)]
    d = process_page(files[0])
    for file in tqdm.notebook.tqdm(files[1:]):
        processed_file = process_page(file)
        if processed_file: 
            combine(d, processed_file)
    
    print("Processed", d["docs"], "docs, resulting in", d["tokens"], "tokens.")
    return d
        
processed = process_dir(path)

In [58]:
with open("nos.pkl", "wb") as file:
    pickle.dump(processed, file)